Задача - получить бейслайн при поможи DNN (улучшить random forest)

In [25]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

%matplotlib inline

In [26]:
df = pd.read_csv('data/titanic/train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [27]:
y = df['Survived']
X = df.drop('Survived', axis = 1)
X_tr, X_test,  y_tr, y_test = train_test_split(X, y, train_size=0.8)
X.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [28]:
def to_float(x):
    def f(x):
        return pd.to_numeric(x, downcast='float')
    return f(x)
    

def prepare_dataframe(df):
    res = pd.get_dummies(df.iloc[:, [1, 3, 4]]).fillna(0)
    res = res.fillna({'Age': df['Age'].median()}).drop(['Sex_female'], axis=1)\
                    .rename(columns={'Sex_male':'Sex'})
    res['Sex'] = res['Sex'] - (res['Sex'].max() - res['Sex'].min()) / 2
    res['Age'] = res['Age'] - (res['Age'].max() - res['Age'].min()) / 2
    res['Pclass'] = res['Pclass'] - (res['Pclass'].max() - res['Pclass'].min()) / 2
    return res.apply(to_float)

X_tr = prepare_dataframe(X_tr)
X_test = prepare_dataframe(X_test)



In [31]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape = (3,)),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dense(16, activation = 'relu'),
    tf.keras.layers.Dense(8, activation = 'relu'),
    tf.keras.layers.Dense(4, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model.compile(optimizer= tf.keras.optimizers.RMSprop(lr=0.003), 
              loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_tr, y_tr, epochs = 60, validation_data=(X_test, y_test), verbose=1)

Epoch 1/60
23/23 [==============================] - 0s 21ms/step - loss: 0.6896 - accuracy: 0.6067 - val_loss: 0.6850 - val_accuracy: 0.5810
Epoch 2/60
23/23 [==============================] - 0s 10ms/step - loss: 0.6611 - accuracy: 0.6264 - val_loss: 0.6734 - val_accuracy: 0.5978
Epoch 3/60
23/23 [==============================] - 0s 9ms/step - loss: 0.6444 - accuracy: 0.6222 - val_loss: 0.6851 - val_accuracy: 0.5978
Epoch 4/60
23/23 [==============================] - 0s 8ms/step - loss: 0.5901 - accuracy: 0.7121 - val_loss: 0.6278 - val_accuracy: 0.7207
Epoch 5/60
23/23 [==============================] - 0s 7ms/step - loss: 0.5726 - accuracy: 0.7191 - val_loss: 0.5197 - val_accuracy: 0.7318
Epoch 6/60
23/23 [==============================] - 0s 7ms/step - loss: 0.5430 - accuracy: 0.7626 - val_loss: 0.5911 - val_accuracy: 0.7095
Epoch 7/60
23/23 [==============================] - 0s 8ms/step - loss: 0.5303 - accuracy: 0.7795 - val_loss: 0.5061 - val_accuracy: 0.7765
Epoch 8/60
23/23 [

Epoch 59/60
23/23 [==============================] - 0s 10ms/step - loss: 0.4425 - accuracy: 0.7851 - val_loss: 0.4370 - val_accuracy: 0.7765
Epoch 60/60
23/23 [==============================] - 0s 6ms/step - loss: 0.4402 - accuracy: 0.7992 - val_loss: 0.4632 - val_accuracy: 0.8101


In [32]:
X_t1 = pd.read_csv('data/titanic/test.csv')
uid = X_t1['PassengerId']
X_t = prepare_dataframe(X_t1)


In [33]:
pred = pd.DataFrame({'PassengerId': uid,
                     'Survived' : model.predict(X_t)[:,0]})

pred['Survived'] = pred['Survived'].apply(lambda x: 0 if x < 0.5 else 1 )  
pred.head()
pred.to_csv('data/titanic/result.csv', index=False)
